# **Image Optimization & DB 2019**
## **1 2019 DataBases**
```
text-table 
```

In [1]:
# //div[contains(@class, 'class1') and contains(@class, 'class2')]
# section_wrap  pc_section  side_basic
# section_bg _section_bg _interactive_bg  fixed_bg

In [2]:
page_index = 9

import requests, re
from lxml.html import fromstring, tostring
regex_url = f"http://arterior-ga.com/{page_index}"
regex_imweb = "^https?://(?:[a-z0-9\-]+\.)+[a-z]{2,6}(?:/[^/#?]+)+\.(?:jpg|gif|png)"
regex_imweb = "https?:\/\/(?:www\.|(?!www))cdn\.imweb.me\/thumbnail\/[a-zA-Z0-9-]+\/[a-zA-Z0-9-]+\.(?:jpg|gif|png)"
regex_image = 'https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.(?:jpg|gif|png)$|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.(?:jpg|gif|png)$'
regex_tokenizer = re.compile(regex_imweb)
def get_image(url:str, xpath:str, get=None):
    result = []
    response      = requests.get(url).content.decode('utf-8')
    response_lxml = fromstring(response) # tostring(_, encoding='unicode')
    response_html = response_lxml.xpath(xpath)
    if get:
        for _html in response_html:
            result.append(_html.get(get))
        return result
    else:
        return response_html

In [3]:
xpath_text = {
    "style":'//div[@doz_type="section"]//div[contains(@class, "_interactive_bg") and contains(@class, "fixed_bg")]',  # 대표 Banner
    "src":"//main/div[@doz_type='inside']//div[@doz_type='widget']/div[@class='_widget_data ']//img[@class=' org_image']", # Problem, Process, After
}

result = []
for k,v in xpath_text.items():
    result += get_image(regex_url, v, k)
result = ["".join(regex_tokenizer.findall(_))  for _ in result]
result

['https://cdn.imweb.me/thumbnail/20200326/da23be369c15f.png',
 'https://cdn.imweb.me/thumbnail/20200324/9261f20884738.jpg',
 'https://cdn.imweb.me/thumbnail/20200325/e1ad0be6bd492.jpg',
 'https://cdn.imweb.me/thumbnail/20200302/741e555076a44.jpg',
 'https://cdn.imweb.me/thumbnail/20200302/7761b0f5227fc.jpg',
 'https://cdn.imweb.me/thumbnail/20200326/58091787124a9.png',
 'https://cdn.imweb.me/thumbnail/20200309/c0b46c55e2c52.jpg']

In [4]:
# Process 이미지 추출하기
get_temp = get_image(regex_url, '//div[@class="widget _gallery_wrap "]')
if len(get_temp) >0:
    text = tostring(get_temp[0]).decode('utf-8')
else: text = ''
list(set(regex_tokenizer.findall(text)))

['https://cdn.imweb.me/thumbnail/20200326/6e0672c8d1a8c.png',
 'https://cdn.imweb.me/thumbnail/20200326/48db0a7566e57.png',
 'https://cdn.imweb.me/thumbnail/20200326/dac70c5dc8da5.png',
 'https://cdn.imweb.me/thumbnail/20200326/35fb6bfa7304c.png',
 'https://cdn.imweb.me/thumbnail/20200326/62539c120e524.png',
 'https://cdn.imweb.me/thumbnail/20200326/e34163eae5c13.png',
 'https://cdn.imweb.me/thumbnail/20200326/9a871b0a25ceb.png',
 'https://cdn.imweb.me/thumbnail/20200326/38d6a414df5fd.png']

## **2 Building to the Function**

In [5]:
page_index = 23

import requests, re
from lxml.html import fromstring, tostring
regex_url = f"http://arterior-ga.com/{page_index}"
regex_imweb = "^https?://(?:[a-z0-9\-]+\.)+[a-z]{2,6}(?:/[^/#?]+)+\.(?:jpg|gif|png)"
regex_imweb = "https?:\/\/(?:www\.|(?!www))cdn\.imweb.me\/thumbnail\/[a-zA-Z0-9-]+\/[a-zA-Z0-9-]+\.(?:jpg|gif|png)"
regex_image = 'https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.(?:jpg|gif|png)$|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.(?:jpg|gif|png)$'
regex_tokenizer = re.compile(regex_imweb)

def get_image(url:str, xpath:str, get=None):
    result = []
    response      = requests.get(url).content.decode('utf-8')
    response_lxml = fromstring(response) # tostring(_, encoding='unicode')
    response_html = response_lxml.xpath(xpath)
    if get:
        for _html in response_html:
            result.append(_html.get(get))
        return result
    else:
        return response_html

def get_image_urls(page_index):
    result_dict = {}
    xpath_text = {
        "style":'//div[@doz_type="section"]//div[contains(@class, "_interactive_bg") and contains(@class, "fixed_bg")]',  # 대표 Banner
        "src":"//main/div[@doz_type='inside']//div[@doz_type='widget']/div[@class='_widget_data ']//img[@class=' org_image']", # Problem, Process, After
    }

    result = []
    for k,v in xpath_text.items():
        result += get_image(regex_url, v, k)
    result = ["".join(regex_tokenizer.findall(_))  for _ in result]
    result_dict['main'] = result

    # Process 이미지 추출하기
    get_temp = get_image(regex_url, '//div[@class="widget _gallery_wrap "]')
    if len(get_temp) >0:
        text = tostring(get_temp[0]).decode('utf-8')
    else: text = ''
    # return result + list(set(regex_tokenizer.findall(text)))
    result_dict['after'] = list(set(regex_tokenizer.findall(text)))
    return result_dict

In [6]:
get_image_urls(2)

{'main': ['https://cdn.imweb.me/thumbnail/20200326/b9b699e781255.png',
  'https://cdn.imweb.me/thumbnail/20200325/d577d36491c33.jpg',
  'https://cdn.imweb.me/thumbnail/20200325/8eae0ad648473.png',
  'https://cdn.imweb.me/thumbnail/20200325/1087a12d8dc5f.png',
  'https://cdn.imweb.me/thumbnail/20200325/2fab64e3b2141.png',
  'https://cdn.imweb.me/thumbnail/20200326/d01e17710ae2e.png',
  'https://cdn.imweb.me/thumbnail/20200310/9e670219333a9.jpg'],
 'after': ['https://cdn.imweb.me/thumbnail/20200326/0d73c2e7c2563.png',
  'https://cdn.imweb.me/thumbnail/20200326/4956ffc730e67.png',
  'https://cdn.imweb.me/thumbnail/20200326/5b67535616f17.png',
  'https://cdn.imweb.me/thumbnail/20200326/f5a2b09b56f9d.png',
  'https://cdn.imweb.me/thumbnail/20200326/d936bf6049980.png',
  'https://cdn.imweb.me/thumbnail/20200326/96c734eaeb394.png']}

## **3 Download And Change the File name**

In [ ]:
test_link = 'https://cdn.imweb.me/thumbnail/20200326/0d73c2e7c2563.png'

import re, os
def change_file_name(link, rename=None):
    r"""Image Links Change to 'rename' file"""
    # Download the Link file
    os.system(f"wget {link}")

    # Get the File name components
    tokenizer = re.compile("[A-z0-9]+\.(?:jpg|gif|png)")
    file_full_name = "".join(tokenizer.findall(text_test))
    file_name = file_full_name.split('.')[0]
    file_format = file_full_name.split('.')[-1]
    
    # Changing the File name
    file_full_name = file_full_name.replace('.', '\.') # 원본 regex 문법에 맞게 변경
    if rename:
        rename = rename + '\.' + file_format   # 대상 확장자 추가 및 regex 문법에 맞게 변경
        os.system(f"mv {file_full_name} {rename}")
    else:
        pass